# Sinir Ağı zihniyetiyle Lojistik Regresyon

Bu çalışma dosyasında, kedileri tanımak için bir lojistik regresyon sınıflandırıcısı oluşturacaksınız. Bir Sinir Ağı zihniyetiyle bunu nasıl yapacağınız konusunda size adım atacak ve ayrıca derin öğrenme hakkındaki sezgilerinizi geliştirecek.

**Bu çalışma dosyasında neler öğrenilecek:**
- Aşağıdakiler dahil bir öğrenme algoritmasının genel mimarisini oluşturma:
     - Başlatma parametreleri
     - Maliyet fonksiyonunun ve gradyanının hesaplanması
     - Bir optimizasyon algoritması kullanma (gradyan iniş)
- Yukarıdaki üç işlevi de doğru sırayla bir ana model fonksiyonu altında toplama.

## 1 - Paketler ##

Öncelikle, bu atama sırasında ihtiyaç duyacağınız tüm paketleri içe aktarmak için aşağıdaki hücreyi çalıştıralım.
- [numpy](www.numpy.org), Python ile bilimsel hesaplama için temel pakettir.
- [h5py](http://www.h5py.org), bir H5 dosyasında depolanan bir veri kümesiyle etkileşim kurmak için yaygın bir pakettir.
- [matplotlib](http://matplotlib.org), Python'da grafik çizmek için ünlü bir kütüphanedir.
- [PIL](http://www.pythonware.com/products/pil/) ve [scipy](https://www.scipy.org/) burada sonunda kendi resminizle modelinizi test etmek için kullanılır .

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import scipy
import scipy.misc
from PIL import Image
from scipy import ndimage
#from lr_utils import load_dataset

%matplotlib inline

In [ ]:
ROOT_DIR = "https://media.githubusercontent.com/media/yapay-ogrenme/casgem-eu-project-training-on-data-mining-2nd/main/PART2/Day9-NeuralNetworks/notebooks/"

DATASET_PATH = ROOT_DIR + "datasets/"

In [ ]:
DATASET_PATH_TRAIN = DATASET_PATH + 'train_catvnoncat_2.h5'
DATASET_PATH_TEST = DATASET_PATH + 'test_catvnoncat_2.h5'

!wget $DATASET_PATH_TRAIN
!wget $DATASET_PATH_TEST

In [ ]:
#@title Load Dataset
def load_dataset():
    train_dataset = h5py.File('train_catvnoncat_2.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels

    test_dataset = h5py.File('test_catvnoncat_2.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels

    classes = np.array(test_dataset["list_classes"][:]) # the list of classes
    
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

## 2 - Probleme Genel Bakış ##

**Problem Açıklaması**: Size aşağıdakileri içeren bir veri kümesi ("data.h5") verilir:
- kedi (y=1) veya kedi olmayan (y=0) olarak etiketlenen m_train görüntülerinden oluşan bir eğitim seti
- kedi veya kedi olmayan olarak etiketlenmiş bir m_test görüntüleri test seti
- her görüntü şekle (num_px, num_px, 3) sahiptir, burada 3, 3 kanal (RGB) içindir. Böylece, her görüntü karedir (yükseklik = num_px) ve (genişlik = num_px).

Resimleri kedi veya kedi olmayan olarak doğru bir şekilde sınıflandırabilen basit bir görüntü tanıma algoritması oluşturacaksınız.

Veri setini daha yakından tanıyalım. Aşağıdaki kodu çalıştırarak verileri yükleyin.

In [ ]:
# Loading the data (cat/non-cat)
train_set_x_orig, train_set_y, test_set_x_orig, test_set_y, classes = load_dataset()

Görüntü veri kümelerinin (eğitim ve test) sonuna "_orig" ekledik çünkü onları ön işlemlere tabi tutacağız. Ön işlemlerden sonra, train_set_x ve test_set_x ile değişkenlerine atayacağız. 
(train_set_y ve test_set_y etiketleri herhangi bir ön işlemeye ihtiyaç duymaz).

train_set_x_orig ve test_set_x_orig öğelerinizin her satırı, bir görüntüyü temsil eden bir dizidir. Aşağıdaki kodu çalıştırarak bir örneği görselleştirebilirsiniz. Ayrıca `index` değerini değiştirmekten çekinmeyin ve diğer resimleri görmek için yeniden çalıştırın.

In [ ]:
# Example of a picture
index = 34
plt.imshow(train_set_x_orig[index])
print ("y = " + str(train_set_y[:,index]) + ", it's a '" + classes[np.squeeze(train_set_y[:,index])].decode("utf-8") +  "' picture.")

**Alıştırma:** Şunun değerlerini bulun:
- m_train (eğitim örneklerinin sayısı)
- m_test (test örneği sayısı)
- num_px (= bir eğitim görüntüsünün yüksekliği = genişliği)

`train_set_x_orig` öğesinin (m_train, num_px, num_px, 3) boyutlarında bir numpy-array olduğunu unutmayın.

Örneğin, `train_set_x_orig.shape[0]` yazarak `m_train`'e erişebilirsiniz.

In [ ]:
### START CODE HERE ### (≈ 3 lines of code)
m_train = train_set_y.shape[1]
m_test = test_set_y.shape[1]
num_px = train_set_x_orig.shape[1]
### END CODE HERE ###

print ("Number of training examples: m_train = " + str(m_train))
print ("Number of testing examples: m_test = " + str(m_test))
print ("Height/Width of each image: num_px = " + str(num_px))
print ("Each image is of size: (" + str(num_px) + ", " + str(num_px) + ", 3)")
print ("train_set_x shape: " + str(train_set_x_orig.shape))
print ("train_set_y shape: " + str(train_set_y.shape))
print ("test_set_x shape: " + str(test_set_x_orig.shape))
print ("test_set_y shape: " + str(test_set_y.shape))

**m_train, m_test ve num_px için Beklenen Çıktı**:
<table style="width:15%">
  <tr>
    <td>m_train</td>
    <td> 209 </td> 
  </tr>
  
  <tr>
    <td>m_test</td>
    <td> 50 </td> 
  </tr>
  
  <tr>
    <td>num_px</td>
    <td> 64 </td> 
  </tr>
  
</table>


Kolaylık sağlamak için, şimdi (num_px, num_px, 3) boyutlarındaki bir resmi, (num_px $*$ num_px $*$ 3, 1) boyutlarında numpy-array olarak yeniden şekillendirmelisiniz. 

Bundan sonra, eğitim (ve test) veri kümemiz, her sütunun düzleştirilmiş (flattened) bir görüntüyü temsil ettiği bir sayısal dizidir. m_train (sırasıyla m_test) sütunları olmalıdır.

**Alıştırma:** Eğitim ve test veri kümelerini, (num_px, num_px, 3) boyutundaki görüntülerin tek boyutlu vektörlerine (num\_px $*$ num\_px $*$ 3, 1) düzleştirileceği şekilde yeniden şekillendirin.

(a,b,c,d) şeklindeki bir X matrisini X_flatten şekilli bir matrise (b$*$c$*$d, a) düzleştirmek istediğinizde kullanabileceğiniz bir hile kullanmaktır:
```python
X_flatten = X.reshape(X.shape[0], -1).T      # X.T is the transpose of X
```

In [ ]:
# Reshape the training and test examples

### START CODE HERE ### (≈ 2 lines of code)
train_set_x_flatten = train_set_x_orig.reshape(train_set_x_orig.shape[0], -1).T
test_set_x_flatten = test_set_x_orig.reshape(test_set_x_orig.shape[0], -1).T
### END CODE HERE ###

print ("train_set_x_flatten shape: " + str(train_set_x_flatten.shape))
print ("train_set_y shape: " + str(train_set_y.shape))
print ("test_set_x_flatten shape: " + str(test_set_x_flatten.shape))
print ("test_set_y shape: " + str(test_set_y.shape))
print ("sanity check after reshaping: " + str(train_set_x_flatten[0:5,0]))

**Beklenen çıktı**:

<table style="width:35%">
  <tr>
    <td>train_set_x_flatten shape</td>
    <td> (12288, 209)</td> 
  </tr>
  <tr>
    <td>train_set_y shape</td>
    <td>(1, 209)</td> 
  </tr>
  <tr>
    <td>test_set_x_flatten shape</td>
    <td>(12288, 50)</td> 
  </tr>
  <tr>
    <td>test_set_y shape</td>
    <td>(1, 50)</td> 
  </tr>
  <tr>
  <td>sanity check after reshaping</td>
  <td>[17 31 56 22 33]</td> 
  </tr>
</table>

Renkli görüntüleri temsil etmek için, her piksel için kırmızı(red), yeşil(green) ve mavi(blue) kanallar (RGB) belirtilmelidir ve bu nedenle piksel değeri aslında 0 ile 255 arasında değişen üç sayıdan oluşan bir vektördür.

Makine öğrenimindeki yaygın bir ön işleme adımı, veri kümenizi ortalamak ve standart hale getirmektir; bu, tüm numpy dizisinin ortalamasını her örnekten çıkarmanız ve ardından her örneği tüm numpy dizisinin standart sapmasına bölmeniz anlamına gelir. Ancak resim veri kümeleri için, daha basit ve daha kullanışlıdır ve veri kümesinin her satırını 255'e (bir piksel kanalının maksimum değeri) bölmek neredeyse işe yarar.

> NOT: Modelinizin eğitimi sırasında, nöron aktivasyonlarını gözlemlemek için ağırlıkları çarpacak ve bazı ilk girdilere önyargılar ekleyeceksiniz. Ardından, modeli eğitmek için gradyanlarla geri yayılırsınız. Ancak, gradyanlarımızın patlamaması için her özelliğin benzer bir aralığa sahip olması son derece önemlidir. Bunu daha sonra derslerde daha ayrıntılı olarak göreceksiniz.

Veri kümemizi standartlaştıralım.

In [ ]:
train_set_x = train_set_x_flatten / 255.
test_set_x = test_set_x_flatten / 255.

<font color='blue'>
**Hatırlamanız gerekenler:**


Yeni bir veri kümesini önceden işlemek için yaygın olarak kullanılan adımlar şunlardır:
- Problemin boyutlarını belirleyin (m_train, m_test, num_px, ...)
- Veri kümelerini, her örnek bir boyutlu vektörü olacak şekilde yeniden boyutlandırma (num_px \* num_px \* 3, 1)
- Verileri "Standardize" etme

## 3 - Öğrenme Algoritmasının Genel Mimarisi ##

Kedi görüntülerini kedi olmayan görüntülerden ayırt etmek için basit bir algoritma tasarlamanın zamanı geldi.

Bir Sinir Ağı zihniyetini kullanarak bir Lojistik Regresyon oluşturacaksınız. Aşağıdaki Şekil, **Lojistik Regresyonun aslında neden çok basit bir Sinir Ağı olduğunu açıklıyor!**

<img src="https://drive.google.com/uc?id=1Pm8ZbTzQzTpHlkTpek8u1KTUn1X0bJZK" style="width:650px;height:400px;" alt="LogReg_kiank" title="LogReg_kiank">

**Algoritmanın matematiksel ifadesi**:

For one example $x^{(i)}$:
$$z^{(i)} = w^T x^{(i)} + b \tag{1}$$
$$\hat{y}^{(i)} = a^{(i)} = sigmoid(z^{(i)})\tag{2}$$ 
$$ \mathcal{L}(a^{(i)}, y^{(i)}) =  - y^{(i)}  \log(a^{(i)}) - (1-y^{(i)} )  \log(1-a^{(i)})\tag{3}$$

The cost is then computed by summing over all training examples:
$$ J = \frac{1}{m} \sum_{i=1}^m \mathcal{L}(a^{(i)}, y^{(i)})\tag{6}$$

**Temel adımlar**:
Bu alıştırmada aşağıdaki adımları gerçekleştireceksiniz:
- Modelin parametrelerini başlatma
- Maliyeti en aza indirerek modelin parametrelerini öğrenme
- Tahmin yapmak için öğrenilen parametreleri kullanma (test setinde)
- Sonuçları analiz etme ve sonuca varma

## 4 - Algoritmamızın parçalarını oluşturmak ##

Bir Sinir Ağı oluşturmanın ana adımları şunlardır:
1. Model yapısını tanımlayın (giriş özelliklerinin sayısı gibi)
2. Modelin parametrelerini başlatın
3. Döngü:
- Yitim (kayıp) (loss) değerini hesaplayın (ileri yayılım) (forward propagation)
- Mevcut gradyanı hesaplayın (geriye doğru yayılma) (backward propagation)
- Parametreleri güncelle (gradyan iniş) (gradient descent)

Genellikle 1-3'ü ayrı ayrı oluşturur ve bunları `model()` dediğimiz tek bir fonksiyına entegre edersiniz.

### 4.1 - Yardımcı işlevler

**Alıştırma**: `sigmoid()` fonksiyonunu tanımlayın ve uygulayın. 
Yukarıdaki şekilde gördüğünüz gibi, tahmin yapabilmek için $sigmoid( w^T x + b)$ hesaplamanız gerekiyor.

In [ ]:
# GRADED FUNCTION: sigmoid

def sigmoid(z):
    """
    Compute the sigmoid of z

    Arguments:
    x -- A scalar or numpy array of any size.

    Return:
    s -- sigmoid(z)
    """

    ### START CODE HERE ### (≈ 1 line of code)
    s = 1 / (1 + np.exp(-z))
    ### END CODE HERE ###
    
    return s

In [ ]:
print ("sigmoid(0) = " + str(sigmoid(0)))
print ("sigmoid(9.2) = " + str(sigmoid(9.2)))

**Beklenen Çıktı**: 

<table style="width:20%">
  <tr>
    <td>sigmoid(0)</td>
    <td> 0.5</td> 
  </tr>
  
  <tr>
    <td>sigmoid(9.2)</td>
    <td> 0.999898970806 </td> 
  </tr>
</table>

### 4.2 - Parametreleri başlatma

**Alıştırma:** Aşağıdaki hücrede parametre başlatmayı uygulayın. w'yi sıfır vektörü olarak başlatmanız gerekir. Hangi numpy işlevini kullanacağınızı bilmiyorsanız, Numpy kitaplığının belgelerinde np.zeros() öğesine bakın.

In [ ]:
# GRADED FUNCTION: initialize_with_zeros

def initialize_with_zeros(dim):
    """
    This function creates a vector of zeros of shape (dim, 1) for w and initializes b to 0.
    
    Argument:
    dim -- size of the w vector we want (or number of parameters in this case)
    
    Returns:
    w -- initialized vector of shape (dim, 1)
    b -- initialized scalar (corresponds to the bias)
    """
    
    ### START CODE HERE ### (≈ 1 line of code)
    w = np.zeros(shape=(dim, 1))
    b = 0
    ### END CODE HERE ###

    assert(w.shape == (dim, 1))
    assert(isinstance(b, float) or isinstance(b, int))
    
    return w, b

In [ ]:
dim = 2
w, b = initialize_with_zeros(dim)
print ("w = " + str(w))
print ("b = " + str(b))

**Beklenen Çıktı**: 
<table style="width:15%">
    <tr>
        <td>w</td>
        <td> [[ 0.]
 [ 0.]] </td>
    </tr>
    <tr>
        <td>b</td>
        <td> 0 </td>
    </tr>
</table>

Görüntü girişleri için w (num_px $\times$ num_px $\times$ 3, 1) boyutlarında olacaktır.

### 4.3 - İleri ve Geri Yayılım

Artık parametreleriniz başlatıldığına göre, parametreleri öğrenmek için "ileri" ve "geri" yayılma adımlarını yapabilirsiniz.

**Alıştırma:** Maliyet fonksiyonunu (cost function) ve gradyanını hesaplayan bir `propagate()` fonksiyonunu uygulayın.

**İpuçları**:

İleri Yayılım:
- X alınız
- $A = \sigma(w^T X + b) = (a^{(0)}, a^{(1)}, ..., a^{(m-1)}, a^{(m)})$ değerini hesaplayınız
- Maliyet fonksiyonunu hesaplayınız: $J = -\frac{1}{m}\sum_{i=1}^{m}y^{(i)}\log(a^{(i)})+(1-y^{(i)})\log(1-a^{(i)})$

İşte kullanacağınız iki formül:

$$ \frac{\partial J}{\partial w} = \frac{1}{m}X(A-Y)^T\tag{7}$$
$$ \frac{\partial J}{\partial b} = \frac{1}{m} \sum_{i=1}^m (a^{(i)}-y^{(i)})\tag{8}$$

In [ ]:
# GRADED FUNCTION: propagate

def propagate(w, b, X, Y):
    """
    Implement the cost function and its gradient for the propagation explained above

    Arguments:
    w -- weights, a numpy array of size (num_px * num_px * 3, 1)
    b -- bias, a scalar
    X -- data of size (num_px * num_px * 3, number of examples)
    Y -- true "label" vector (containing 0 if non-cat, 1 if cat) of size (1, number of examples)

    Return:
    cost -- negative log-likelihood cost for logistic regression
    dw -- gradient of the loss with respect to w, thus same shape as w
    db -- gradient of the loss with respect to b, thus same shape as b
    
    Tips:
    - Write your code step by step for the propagation
    """
    
    m = X.shape[1]
    
    # FORWARD PROPAGATION (FROM X TO COST)
    ### START CODE HERE ### (≈ 2 lines of code)
    A = sigmoid(np.dot(w.T, X) + b)  # compute activation
    cost = (- 1 / m) * np.sum(Y * np.log(A) + (1 - Y) * (np.log(1 - A)))  # compute cost
    ### END CODE HERE ###
    
    # BACKWARD PROPAGATION (TO FIND GRAD)
    ### START CODE HERE ### (≈ 2 lines of code)
    dw = (1 / m) * np.dot(X, (A - Y).T)
    db = (1 / m) * np.sum(A - Y)
    ### END CODE HERE ###

    assert(dw.shape == w.shape)
    assert(db.dtype == float)
    cost = np.squeeze(cost)
    assert(cost.shape == ())
    
    grads = {"dw": dw,
             "db": db}
    
    return grads, cost

In [ ]:
w, b, X, Y = np.array([[1], [2]]), 2, np.array([[1,2], [3,4]]), np.array([[1, 0]])

grads, cost = propagate(w, b, X, Y)

print ("dw = " + str(grads["dw"]))
print ("db = " + str(grads["db"]))
print ("cost = " + str(cost))

**Beklenen Çıktı**:

<table style="width:50%">
    <tr>
        <td>dw</td>
        <td> [[ 0.99993216]
 [ 1.99980262]]</td>
    </tr>
    <tr>
        <td>db</td>
        <td> 0.499935230625 </td>
    </tr>
    <tr>
        <td>cost</td>
        <td> 6.000064773192205</td>
    </tr>

</table>

### d) Optimizasyon
- Parametrelerinizi başlattınız.
- Ayrıca bir maliyet fonksiyonunu ve onun gradyanını da hesaplayabilirsiniz.
- Şimdi, gradyan inişini kullanarak parametreleri güncellemek istiyorsunuz.

**Alıştırma:** Optimizasyon fonksiyonunu yazın. Amaç, maliyet fonksiyonunu $J$'ı minimize ederek $w$ ve $b$'ı öğrenmektir. $\theta$ parametresi için güncelleme kuralı $ \theta = \theta - \alpha \text{ } d\theta$'dır, burada $\alpha$ öğrenme oranıdır.

In [ ]:
# GRADED FUNCTION: optimize

def optimize(w, b, X, Y, num_iterations, learning_rate, print_cost = False):
    """
    This function optimizes w and b by running a gradient descent algorithm
    
    Arguments:
    w -- weights, a numpy array of size (num_px * num_px * 3, 1)
    b -- bias, a scalar
    X -- data of shape (num_px * num_px * 3, number of examples)
    Y -- true "label" vector (containing 0 if non-cat, 1 if cat), of shape (1, number of examples)
    num_iterations -- number of iterations of the optimization loop
    learning_rate -- learning rate of the gradient descent update rule
    print_cost -- True to print the loss every 100 steps
    
    Returns:
    params -- dictionary containing the weights w and bias b
    grads -- dictionary containing the gradients of the weights and bias with respect to the cost function
    costs -- list of all the costs computed during the optimization, this will be used to plot the learning curve.
    
    Tips:
    You basically need to write down two steps and iterate through them:
        1) Calculate the cost and the gradient for the current parameters. Use propagate().
        2) Update the parameters using gradient descent rule for w and b.
    """
    
    costs = []
    
    for i in range(num_iterations):
        
        
        # Cost and gradient calculation (≈ 1-4 lines of code)
        ### START CODE HERE ### 
        grads, cost = propagate(w, b, X, Y)
        ### END CODE HERE ###
        
        # Retrieve derivatives from grads
        dw = grads["dw"]
        db = grads["db"]
        
        # update rule (≈ 2 lines of code)
        ### START CODE HERE ###
        w = w - learning_rate * dw  # need to broadcast
        b = b - learning_rate * db
        ### END CODE HERE ###
        
        # Record the costs
        if i % 100 == 0:
            costs.append(cost)
        
        # Print the cost every 100 training examples
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" % (i, cost))
    
    params = {"w": w,
              "b": b}
    
    grads = {"dw": dw,
             "db": db}
    
    return params, grads, costs

In [ ]:
params, grads, costs = optimize(w, b, X, Y, num_iterations= 100, learning_rate = 0.009, print_cost = False)

print ("w = " + str(params["w"]))
print ("b = " + str(params["b"]))
print ("dw = " + str(grads["dw"]))
print ("db = " + str(grads["db"]))

**Alıştırma:** Önceki fonksiyon, öğrenilen w ve b'yi verir. Bir X veri kümesinin etiketlerini tahmin etmek için w ve b'yi kullanabiliriz. 

`predict()` fonksiyonunu uygulayın. Tahminleri hesaplamanın iki adımı vardır:

1. $\hat{Y} = A = \sigma(w^T X + b)$ hesaplayın

2. a'nın girişlerini 0'a (aktivasyon <= 0,5 ise) veya 1'e (aktivasyon > 0,5 ise) dönüştürün, tahminleri bir `Y_prediction` vektöründe depolar. İsterseniz, bir `for` döngüsünde `if`/`else` ifadesini kullanabilirsiniz (ancak bunu vektörleştirmenin bir yolu da vardır).

In [ ]:
# GRADED FUNCTION: predict

def predict(w, b, X):
    '''
    Predict whether the label is 0 or 1 using learned logistic regression parameters (w, b)
    
    Arguments:
    w -- weights, a numpy array of size (num_px * num_px * 3, 1)
    b -- bias, a scalar
    X -- data of size (num_px * num_px * 3, number of examples)
    
    Returns:
    Y_prediction -- a numpy array (vector) containing all predictions (0/1) for the examples in X
    '''
    
    m = X.shape[1]
    Y_prediction = np.zeros((1, m))
    w = w.reshape(X.shape[0], 1)
    
    # Compute vector "A" predicting the probabilities of a cat being present in the picture
    ### START CODE HERE ### (≈ 1 line of code)
    A = sigmoid(np.dot(w.T, X) + b)
    ### END CODE HERE ###
    
    for i in range(A.shape[1]):
        # Convert probabilities a[0,i] to actual predictions p[0,i]
        ### START CODE HERE ### (≈ 4 lines of code)
        Y_prediction[0, i] = 1 if A[0, i] > 0.5 else 0
        ### END CODE HERE ###
    
    assert(Y_prediction.shape == (1, m))
    
    return Y_prediction

In [ ]:
print("predictions = " + str(predict(w, b, X)))

**Beklenen Çıktı**: 

<table style="width:30%">
    <tr>
         <td>
            predictions
         </td>
          <td>
            [[ 1.  1.]]
         </td>  
   </tr>

</table>


<font color='blue'>
**Hatırlanması gerekenler:**

Şunları yapan birkaç fonksiyonu uyguladınız:

- Initialize (w,b)
- Parametreleri (w,b) öğrenmek için kaybı yinelemeli olarak optimize edin:
  - maliyetin ve gradyanının hesaplanması
  - gradyan iniş kullanarak parametreleri güncelleme
- Verilen bir dizi örnek için etiketleri tahmin etmek için öğrenilenleri (w,b) kullanın

## 5 - Tüm fonksiyonları bir modelde birleştirin ##

Şimdi tüm yapı taşlarını (önceki bölümlerde uygulanan fonksiyonlar) doğru sırayla bir araya getirerek genel modelin nasıl yapılandırıldığını göreceksiniz.

**Alıştırma:** 
Model fonksiyonunu uygulayın. Aşağıdaki gösterimi kullanın:
- Test kümesindeki tahminleriniz için Y_prediction
- Tren kümesindeki tahminleriniz için Y_prediction_train
- optimize() fonksiyonun çıktıları: w, costs, grads

In [ ]:
# GRADED FUNCTION: model

def model(X_train, Y_train, X_test, Y_test, num_iterations=2000, learning_rate=0.5, print_cost=False):
    """
    Builds the logistic regression model by calling the function you've implemented previously
    
    Arguments:
    X_train -- training set represented by a numpy array of shape (num_px * num_px * 3, m_train)
    Y_train -- training labels represented by a numpy array (vector) of shape (1, m_train)
    X_test -- test set represented by a numpy array of shape (num_px * num_px * 3, m_test)
    Y_test -- test labels represented by a numpy array (vector) of shape (1, m_test)
    num_iterations -- hyperparameter representing the number of iterations to optimize the parameters
    learning_rate -- hyperparameter representing the learning rate used in the update rule of optimize()
    print_cost -- Set to true to print the cost every 100 iterations
    
    Returns:
    d -- dictionary containing information about the model.
    """
    
    ### START CODE HERE ###
    # initialize parameters with zeros (≈ 1 line of code)
    w, b = initialize_with_zeros(X_train.shape[0])

    # Gradient descent (≈ 1 line of code)
    parameters, grads, costs = optimize(w, b, X_train, Y_train, num_iterations, learning_rate, print_cost)
    
    # Retrieve parameters w and b from dictionary "parameters"
    w = parameters["w"]
    b = parameters["b"]
    
    # Predict test/train set examples (≈ 2 lines of code)
    Y_prediction_test = predict(w, b, X_test)
    Y_prediction_train = predict(w, b, X_train)

    ### END CODE HERE ###

    # Print train/test Errors
    print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
    print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))

    
    d = {"costs": costs,
         "Y_prediction_test": Y_prediction_test, 
         "Y_prediction_train" : Y_prediction_train, 
         "w" : w, 
         "b" : b,
         "learning_rate" : learning_rate,
         "num_iterations": num_iterations}
    
    return d

Modelinizi eğitmek için aşağıdaki kod bloğunu çalıştırın.

In [ ]:
d = model(train_set_x, train_set_y, test_set_x, test_set_y, num_iterations = 2000, learning_rate = 0.005, print_cost = True)

**Beklenen Çıktı**: 

<table style="width:40%"> 
    <tr>
        <td> Train Accuracy </td> 
        <td> 99.04306220095694 % </td>
    </tr>
    <tr>
        <td>Test Accuracy </td> 
        <td> 70.0 % </td>
    </tr>
</table>


**Yorum**: Eğitim doğruluğu %100'e yakındır. Modeliniz çalışıyor ve eğitim verilerine uyacak kadar yüksek kapasiteye sahip. Test hatası %68'dir. Kullandığımız küçük veri kümesi ve lojistik regresyonun doğrusal bir sınıflandırıcı olduğu düşünüldüğünde, aslında bu basit model için fena değil. 

Ayrıca, modelin eğitim verilerine aşırı uyduğunu görüyorsunuz. Bu uzmanlıkta daha sonra, örneğin düzenlileştirmeyi kullanarak aşırı uymayı nasıl azaltacağınızı öğreneceksiniz. Aşağıdaki kodu kullanarak (ve `index` değişkenini değiştirerek) test kümesindeki resimlerindeki tahminlere bakabilirsiniz.

In [ ]:
# Example of a picture that was wrongly classified.
index = 1
plt.imshow(test_set_x[:,index].reshape((num_px, num_px, 3)))
print ("y = " + str(test_set_y[0, index]) + ", you predicted that it is a \"" + classes[int(d["Y_prediction_test"][0, index])].decode("utf-8") +  "\" picture.")

Ayrıca maliyet fonksiyonunu ve gradyanları da çizelim.

In [ ]:
# Plot learning curve (with costs)
costs = np.squeeze(d['costs'])
plt.plot(costs)
plt.ylabel('cost')
plt.xlabel('iterations (per hundreds)')
plt.title("Learning rate =" + str(d["learning_rate"]))
plt.show()

Maliyetin düştüğünü görebilirsiniz. Parametrelerin öğrenildiğini gösterir. Ancak eğitim kümesinde modeli daha da fazla eğitebileceğinizi görüyorsunuz. Yukarıdaki hücredeki yineleme sayısını artırmaya çalışın ve hücreleri yeniden çalıştırın. Eğitim kümesinde doğruluğunun arttığını ancak test seti doğruluğunun düştüğünü görebilirsiniz. Buna aşırı uyum (overfitting) denir.

## 6 - İleri analiz ##

İlk görüntü sınıflandırma modelinizi oluşturduğunuz için tebrikler. Bunu biraz daha analiz edelim ve $\alpha$ öğrenme oranı için olası seçenekleri inceleyelim.

#### Öğrenme oranı seçimi ####

**Hatırlatma**:
Gradient Descent'in çalışması için öğrenme oranını akıllıca seçmelisiniz. $\alpha$ öğrenme oranı, parametreleri ne kadar hızlı güncellediğimizi belirler. Öğrenme oranı çok büyükse, optimal değeri "aşabiliriz (overshoot)". Benzer şekilde, eğer çok küçükse, en iyi değerlere yakınsamak için çok fazla iterasyona ihtiyacımız olacaktır. Bu yüzden iyi ayarlanmış bir öğrenme oranı kullanmak çok önemlidir.

Modelimizin öğrenme eğrisini çeşitli öğrenme oranları seçenekleriyle karşılaştıralım. Aşağıdaki hücreyi çalıştırın. Bu yaklaşık 1 dakika sürmelidir. Ayrıca, içermesi için `learning_rates` değişkenini başlattığımız üç değerden farklı değerler denemekten çekinmeyin ve ne olduğunu görün.

In [ ]:
learning_rates = [0.01, 0.001, 0.0001]
models = {}
for i in learning_rates:
    print ("learning rate is: " + str(i))
    models[str(i)] = model(train_set_x, train_set_y, test_set_x, test_set_y, num_iterations = 1500, learning_rate = i, print_cost = False)
    print ('\n' + "-------------------------------------------------------" + '\n')

for i in learning_rates:
    plt.plot(np.squeeze(models[str(i)]["costs"]), label= str(models[str(i)]["learning_rate"]))

plt.ylabel('cost')
plt.xlabel('iterations')

legend = plt.legend(loc='upper center', shadow=True)
frame = legend.get_frame()
frame.set_facecolor('0.90')
plt.show()

- Farklı öğrenme oranları, farklı maliyetler ve dolayısıyla farklı tahmin sonuçları verir.
- Öğrenme oranı çok büyükse (0.01), maliyet yukarı ve aşağı salınım yapabilir. Hatta farklılaşabilir (bu örnekte, 0.01'in kullanılması yine de sonunda maliyet için iyi bir değerde sonuçlanır).
- Daha düşük maliyet, daha iyi bir model anlamına gelmez. Aşırı uyum olup olmadığını kontrol etmelisiniz. Eğitim doğruluğu, test doğruluğundan çok daha yüksek olduğunda olur.

- Derin öğrenmede genellikle şunları yapmanızı öneririz:
  - Maliyet fonksiyonunu daha iyi en aza indiren öğrenme oranını seçin.
  - Modeliniz aşırı uyum gösteriyorsa, aşırı uyumu azaltma tekniklerini kullanın.